This is where I will process my data and run a model

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras import layers

import cv2

from get_coords import Data

# do batch prediction function DONE
# mix features together 1/2 DONE
# add color features DONE
# mean square error for regression DONE
# get started writing 
# get results 1/2 DONE

# Features to try: DONE
# 1) Raw Position Values DONE
# Find how they did visual complexity 
# Think about why some features are doing better
# try cross validation 
# reshuffle training and testing data
# try removing some percent of training and test on different testing sets and train on different training sets
# train should be different from test ALWAYS

# start writing related works 
# start writing methodology
# write indroduction
# write like other papers that I have read

# try to get positive results

# arxiv for posting negative result


Process Data

In [2]:
classes = ['button', 'field', 'heading', 'iframe', 'image', 'label', 'link', 'text']

directory = "dataset/images/english/"
#directory = "small_test/"

data = Data(
    directory=directory
    )

raw_data = data.filter_confidence()
image_names = list(raw_data.keys())
image_path = data.images()
images = {}


for i in range(len(image_path)):
    images[image_names[i]] = cv2.imread(image_path[i])

def raw():
    rawData = {}
    temp = list(raw_data.values())
    for i in range(len(temp)):
        temp_rawData = []
        for element in temp[i]:
            e = []
            e.append(float((float(element[0])+float(element[2]))/2))
            e.append(float((float(element[1])+float(element[3]))/2))
            e.append(float(element[5]))
            temp_rawData.extend(e)
        rawData[int(image_names[i])] = temp_rawData
    sizes = []
    for i in range(len(rawData)):
        sizes.append(len(rawData[i]))
    max_size = max(sizes)
    for i in range(len(rawData)):
        if len(rawData[i]) < max_size:
            for j in range(max_size - len(rawData[i])):
                rawData[i].append(0)
    return rawData

#print(raw())


def colorfulness():
    colorfulness = {}
    for i in range(len(list(images.values()))):
        current_colorfulness = 0
        count = 0
        temp_image = list(images.values())[i] 
        hsv = cv2.cvtColor(temp_image, cv2.COLOR_BGR2HSV)
        for row in range(len(hsv)):
            for column in range(len(hsv[row])):
                current_colorfulness = current_colorfulness + hsv[row][column][1] * hsv[row][column][2]
                count = count + 1
        result = current_colorfulness/count
        colorfulness[int(list(images.keys())[i])] = result
    return colorfulness

def avgPos():
    temp = list(raw_data.values())
    avg_element_position = {}
    for i in range(len(temp)):
        avg_pixel_position_for_class = {}
        count = {}
        for element in temp[i]:
            if avg_pixel_position_for_class.get(int(element[5])) == None:
                avg_pixel_position_for_class[int(element[5])] = ((float(element[0])+float(element[2]))/2,(float(element[1])+float(element[3]))/2)
                count[int(element[5])] = 1
            else:
                count[int(element[5])] += 1
                avg_pixel_position_for_class[int(element[5])] = ((((float(element[0])+float(element[2]))/2)+avg_pixel_position_for_class[int(element[5])][0])/count[int(element[5])],(((float(element[1])+float(element[3]))/2)+avg_pixel_position_for_class[int(element[5])][0])/count[int(element[5])])
                
        avg_element_position[int(image_names[i])] = avg_pixel_position_for_class
        #print(count)

    return avg_element_position

def freqQuad():
    class_per_quadrant = {}
    temp = list(raw_data.values())
    for i in range(len(temp)):
        temp_data_holder = [{},{},{},{}]
        temp_data_holder_2 = []
        temp_image_name = image_names[i]
        temp_image = cv2.imread(directory + temp_image_name + '.png')
        w = temp_image.shape[0]
        h = temp_image.shape[1]
        for element in temp[i]:
            center_X = (element[0]+element[2])/2
            center_Y = (element[1]+element[3])/2
            if center_X >= w/2 and center_Y >= h/2:
                if temp_data_holder[0].get(int(element[5])) == None:
                    temp_data_holder[0][int(element[5])] = 1
                else:
                    temp_data_holder[0][int(element[5])] = temp_data_holder[0][int(element[5])] + 1
            if center_X < w/2 and center_Y >= h/2:
                if temp_data_holder[1].get(int(element[5])) == None:
                    temp_data_holder[1][int(element[5])] = 1
                else:
                    temp_data_holder[1][int(element[5])] = temp_data_holder[1][int(element[5])] + 1
            if center_X < w/2 and center_Y < h/2:
                if temp_data_holder[2].get(int(element[5])) == None:
                    temp_data_holder[2][int(element[5])] = 1
                else:
                    temp_data_holder[2][int(element[5])] = temp_data_holder[2][int(element[5])] + 1
            if center_X >= w/2 and center_Y < h/2:
                if temp_data_holder[3].get(int(element[5])) == None:
                    temp_data_holder[3][int(element[5])] = 1
                else:
                    temp_data_holder[3][int(element[5])] = temp_data_holder[3][int(element[5])] + 1
        for quadrant in temp_data_holder:
            for i in range(8):
                if quadrant.get(i) == None:
                    temp_data_holder_2.append(0)
                else:
                    temp_data_holder_2.append(quadrant[i])
        class_per_quadrant[int(temp_image_name)] = temp_data_holder_2
    return class_per_quadrant
    
def freqClass():
    data = {}
    temp = list(raw_data.values())
    for i in range(len(temp)):
        per_image_data = {}
        for element in temp[i]:
            if per_image_data.get(int(element[5])) == None:
                per_image_data[int(element[5])] = 1
            else:
                per_image_data[int(element[5])] = per_image_data[int(element[5])] + 1
        data[int(image_names[i])] = per_image_data
    return data


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\woprg/.cache\torch\hub\master.zip
YOLOv5  2022-10-13 Python-3.10.4 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
WARNING  NMS time limit 0.550s exceeded


Raw Data

In [14]:
import csv
import numpy as np

path = r"dataset\preprocess\train_means_list.csv"

SVM = False
COLOR = False

normalizer = tf.keras.layers.Normalization(axis=-1)

data = {}

with open(path) as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if "english_resized" in row[0]:
            key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
            value = float(row[1])
            data[key] = value

freq_quad_temp = raw()
freq_quad = []
quad_ratings = []
quad_ratings_binary = []

for key, value in freq_quad_temp.items():
    if not data.get(key) == None:
        freq_quad.append(value)
        quad_ratings.append(data[key])
        if data[key] >= 4:
            quad_ratings_binary.append(1)
        else:
            quad_ratings_binary.append(0)

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(tf.convert_to_tensor(freq_quad))

raw_linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

raw_linear_model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
    loss='mean_squared_logarithmic_error',
    metrics=[tf.keras.metrics.MeanSquaredLogarithmicError()])

print(np.asarray(freq_quad).shape)

history = raw_linear_model.fit(
    tf.convert_to_tensor(freq_quad),
    tf.convert_to_tensor(quad_ratings),
    epochs=100,
    validation_split=0.2,
    # Suppress logging.
    verbose=0)

c:\Users\binad\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


(255, 237)


Raw Data Validation

In [18]:
from os import listdir
from os.path import isfile, join
from sklearn.metrics import mean_squared_error

COLOR = False
errFiles = []

def predictRaw(x):
    d = Data(directory=x)
    raw_data = d.filter_confidence()
    #print(raw_data)
    filename_2 = [f for f in listdir(x) if isfile(join(x, f))]#[0]
    predictions = []
    dataholder = []
    contr = 0
    for filename in filename_2:
        temp = list(raw_data[filename.replace(".png","")])
        rawData = []
        for i in range(len(temp)):
            element = [float(x) for x in list(temp[i])]
            temp_rawData = []
            e = []
            #print ('Element0=%f Element1=%f Element2=%f Element3=%f Element4=%f Element5=%f'%(element[0],element[1],element[2],element[3],element[4],element[5]))
            e.append(float((float(element[0])+float(element[2]))/2))
            e.append(float((float(element[1])+float(element[3]))/2))
            e.append(float(element[5]))
            temp_rawData.extend(e)
            rawData.extend(temp_rawData)
        dataholder.append(rawData)
    for key in range(len(dataholder)):
        if len(dataholder[key]) < 237:
            for j in range(237 - len(dataholder[key])):
                dataholder[key].append(0)
    try:
        predictions = raw_linear_model.predict(tf.convert_to_tensor(dataholder))
    except Exception as e:
        print("Error in invoking raw_linear model. Exception is %s" %(e))
    return predictions
    
predicted = predictRaw("small_test/")
##print(predicted)
data = {}
path = r"dataset\preprocess\train_means_list.csv"
with open(path) as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if "english_resized" in row[0]:
            key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
            value = float(row[1])
            data[key] = value
actual = []
#print(data)
i = 0
files = [f for f in listdir("small_test/") if isfile(join("small_test/", f))]
for f in files:
    
    if (f in errFiles):
        continue
    if not data.get(int(f.replace(".png",""))) == None:
        actual.append(data[int(f.replace(".png",""))])
    else:
        actual.append(-1)
    try:
        print ("FILENAME=%s PREDICATED=%.5f ACTUAL=%.5f" %(f,predicted[i],actual[i]))
    except Exception as e:
        print(":(")
    i+=1
##print(actual)

predicted_2 = []
actual_2 = []

print ('PREDICTED LEN IS %d and ACTUAL LEN IS %d' %(len(predicted),len(actual)))
for i in range(len(actual)):
    if not int(actual[i]) == -1:
        try:
            predicted_2.append(predicted[i])
            actual_2.append(actual[i])
        except Exception as e:
            print ('something went wrong again : Exception is %s' %(e))

print ('Mean Squared Error is %.20f' %(mean_squared_error(actual_2, predicted_2)))

Using cache found in C:\Users\binad/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-9-11 Python-3.10.4 torch-1.12.1+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7031701 parameters, 0 gradients
Adding AutoShape... 


11/11 [==============================] - 0s 5ms/step


c:\Users\binad\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


FILENAME=0.png PREDICATED=-16.49461 ACTUAL=2.23565
FILENAME=1.png PREDICATED=5.43727 ACTUAL=4.18176
FILENAME=10.png PREDICATED=-0.62300 ACTUAL=3.91676
FILENAME=100.png PREDICATED=4.97514 ACTUAL=4.95385
FILENAME=101.png PREDICATED=4.91003 ACTUAL=-1.00000
FILENAME=102.png PREDICATED=4.64351 ACTUAL=4.88532
FILENAME=103.png PREDICATED=4.46681 ACTUAL=5.14240
FILENAME=104.png PREDICATED=3.91548 ACTUAL=4.46853
FILENAME=105.png PREDICATED=-0.98268 ACTUAL=2.26392
FILENAME=106.png PREDICATED=5.20138 ACTUAL=5.39000
FILENAME=107.png PREDICATED=4.97061 ACTUAL=-1.00000
FILENAME=108.png PREDICATED=5.11942 ACTUAL=4.58055
FILENAME=109.png PREDICATED=3.24865 ACTUAL=3.78571
FILENAME=11.png PREDICATED=5.21135 ACTUAL=3.98754
FILENAME=110.png PREDICATED=-10.01425 ACTUAL=-1.00000
FILENAME=111.png PREDICATED=5.21581 ACTUAL=5.79868
FILENAME=112.png PREDICATED=4.60128 ACTUAL=4.51024
FILENAME=113.png PREDICATED=4.67739 ACTUAL=5.14398
FILENAME=114.png PREDICATED=-4.23131 ACTUAL=2.92203
FILENAME=115.png PREDICATED

Quadrant Data Formatting

In [ ]:
import csv
import numpy as np

path = r"dataset\preprocess\train_means_list.csv"

SVM = False
COLOR = True

normalizer = tf.keras.layers.Normalization(axis=-1)

data = {}

with open(path) as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if "english_resized" in row[0]:
            key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
            value = float(row[1])
            data[key] = value

if COLOR:
    c = colorfulness()

freq_quad_temp = freqQuad()
freq_quad = []
quad_ratings = []
quad_ratings_binary = []

for key, value in freq_quad_temp.items():
    if not data.get(key) == None:
        if COLOR:
            value.append(c[key])
        freq_quad.append(value)
        quad_ratings.append(data[key])
        if data[key] >= 4:
            quad_ratings_binary.append(1)
        else:
            quad_ratings_binary.append(0)

#print(freq_quad)
#print(quad_ratings)

Quadrant linear regression

In [13]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(tf.convert_to_tensor(freq_quad))

quad_linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

quad_linear_model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
    loss='mean_squared_logarithmic_error',
    metrics=[tf.keras.metrics.MeanSquaredLogarithmicError()])

history = quad_linear_model.fit(
    tf.convert_to_tensor(freq_quad),
    tf.convert_to_tensor(quad_ratings),
    epochs=100,
    validation_split=0.2,
    # Suppress logging.
    verbose=0)

Quadrant linear regression validation

In [30]:
from os import listdir
from os.path import isfile, join
from sklearn.metrics import mean_squared_error

COLOR = False

def predictQuad(x):
    d = Data(directory=x)
    raw_data = d.filter_confidence()
    #print(raw_data)
    filename_2 = [f for f in listdir(x) if isfile(join(x, f))]#[0]
    predictions = []
    for filename in filename_2:
        shape = cv2.imread(x + str(filename)).shape
        temp = list(raw_data[filename.replace(".png","")])
        for i in range(len(temp)):
            element = [float(x) for x in list(temp[i])]
            temp_data_holder = [{},{},{},{}]
            temp_data_holder_2 = []
            temp_image_name = image_names[i]
            temp_image = cv2.imread(directory + temp_image_name + '.png')
            w = temp_image.shape[0]
            h = temp_image.shape[1]
            center_X = (element[0]+element[2])/2
            center_Y = (element[1]+element[3])/2
            if center_X >= w/2 and center_Y >= h/2:
                if temp_data_holder[0].get(int(element[5])) == None:
                    temp_data_holder[0][int(element[5])] = 1
                else:
                    temp_data_holder[0][int(element[5])] = temp_data_holder[0][int(element[5])] + 1
            if center_X < w/2 and center_Y >= h/2:
                if temp_data_holder[1].get(int(element[5])) == None:
                    temp_data_holder[1][int(element[5])] = 1
                else:
                    temp_data_holder[1][int(element[5])] = temp_data_holder[1][int(element[5])] + 1
            if center_X < w/2 and center_Y < h/2:
                if temp_data_holder[2].get(int(element[5])) == None:
                    temp_data_holder[2][int(element[5])] = 1
                else:
                    temp_data_holder[2][int(element[5])] = temp_data_holder[2][int(element[5])] + 1
            if center_X >= w/2 and center_Y < h/2:
                if temp_data_holder[3].get(int(element[5])) == None:
                    temp_data_holder[3][int(element[5])] = 1
                else:
                    temp_data_holder[3][int(element[5])] = temp_data_holder[3][int(element[5])] + 1
            for quadrant in temp_data_holder:
                for i in range(8):
                    if quadrant.get(i) == None:
                        temp_data_holder_2.append(0)
                    else:
                        temp_data_holder_2.append(quadrant[i])
            if COLOR:
                c = colorfulness()
                temp_data_holder_2.append(c[int(temp_image_name)])
        prediction = quad_linear_model.predict([temp_data_holder_2])
        predictions.append(prediction[0][0])
    return predictions
    
predicted = predictQuad("small_test/")

print(predicted)
data = {}
path = r"dataset\preprocess\train_means_list.csv"
with open(path) as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if "english_resized" in row[0]:
            key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
            value = float(row[1])
            data[key] = value
actual = []
#print(data)
files = [f for f in listdir("small_test/") if isfile(join("small_test/", f))]
for f in files:
    if not data.get(int(f.replace(".png",""))) == None:
        actual.append(data[int(f.replace(".png",""))])
    else:
        actual.append(-1)
print(actual)

predicted_2 = []
actual_2 = []
for i in range(len(actual)):
    if not actual[i] == -1:
        try:
            predicted_2.append(predicted[i])
            actual_2.append(actual[i])
        except Exception as e:
            print("something went wrong again :(")

mean_squared_error(actual_2, predicted_2)


Using cache found in C:\Users\woprg/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-18 Python-3.10.4 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


ValueError: in user code:

    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1791, in predict_step
        return self(x, training=False)
    File "c:\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "normalization_3" (type Normalization).
    
    Dimensions must be equal, but are 32 and 33 for '{{node sequential_2/normalization_3/sub}} = Sub[T=DT_FLOAT](sequential_2

: 

SVM Quadrant

In [12]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler

apply_scaler = True

#print(avg_pos)

if apply_scaler:
    scaler1 = StandardScaler()
    scaler1.fit(freq_quad)
    quad_data = scaler1.transform(freq_quad)
else:
    quad_data = freq_quad

quad_clf = svm.SVC()

quad_clf.fit(quad_data, quad_ratings_binary)

SVC()

Quadrant SVM Validate

In [ ]:
from os import listdir
from os.path import isfile, join

def predictQuadSVM(x):
    d = Data(directory=x)
    raw_data = d.filter_confidence()
    #print(raw_data)
    filename_2 = [f for f in listdir(x) if isfile(join(x, f))]#[0]
    predictions = []
    for filename in filename_2:
        shape = cv2.imread(x + str(filename)).shape
        temp = list(raw_data[filename.replace(".png","")])
        for i in range(len(temp)):
            element = temp[i]
            temp_data_holder = [{},{},{},{}]
            temp_data_holder_2 = []
            temp_image_name = image_names[i]
            temp_image = cv2.imread(directory + temp_image_name + '.png')
            w = temp_image.shape[0]
            h = temp_image.shape[1]
            #print("Width: " + str(w) + " Height: " + str(h))
            center_X = float((element[0]+element[2])/2)
            center_Y = float((element[1]+element[3])/2)
            #print("Position: (" + str(center_X) + ", " + str(center_Y) + ")")
            print(int(element[5]))
            if center_X >= w/2 and center_Y >= h/2:
                print("Top right")
                if temp_data_holder[0].get(int(element[5])) == None:
                    temp_data_holder[0][int(element[5])] = 1
                else:
                    temp_data_holder[0][int(element[5])] = temp_data_holder[0][int(element[5])] + 1
            if center_X < w/2 and center_Y >= h/2:
                print("Top left")
                if temp_data_holder[1].get(int(element[5])) == None:
                    temp_data_holder[1][int(element[5])] = 1
                else:
                    temp_data_holder[1][int(element[5])] = temp_data_holder[1][int(element[5])] + 1
            if center_X < w/2 and center_Y < h/2:
                print("Bottom left")
                if temp_data_holder[2].get(int(element[5])) == None:
                    temp_data_holder[2][int(element[5])] = 1
                else:
                    temp_data_holder[2][int(element[5])] = temp_data_holder[2][int(element[5])] + 1
            if center_X >= w/2 and center_Y < h/2:
                print("Bottom right")
                if temp_data_holder[3].get(int(element[5])) == None:
                    temp_data_holder[3][int(element[5])] = 1
                else:
                    temp_data_holder[3][int(element[5])] = temp_data_holder[3][int(element[5])] + 1
            #print(temp_data_holder)
            for quadrant in temp_data_holder:
                for i in range(8):
                    if quadrant.get(i) == None:
                        temp_data_holder_2.append(0)
                    else:
                        temp_data_holder_2.append(quadrant[i])
            if COLOR:
                c = colorfulness()
                temp_data_holder_2.append(c[int(temp_image_name)])
            try:
                
                #print(temp_data_holder_2)
                prediction = quad_clf.predict([temp_data_holder_2])
                predictions.append(prediction)
            except Exception as e:
                print("Something went wrong but we are gonna pretend like it didn't happen :)")
    return predictions

print(predictQuadSVM(
    #"small_test/"
    "dataset/images/english/"
    ))
data = {}
path = r"dataset\preprocess\train_means_list.csv"
with open(path) as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if "english_resized" in row[0]:
            key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
            value = float(row[1])
            data[key] = value
print(data[int([f for f in listdir("small_test/") if isfile(join("small_test/", f))][0].replace(".png",""))])

Quadrant Random Forest

In [5]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

apply_scaler3 = True

if apply_scaler3:
    scaler3 = StandardScaler()
    scaler3.fit(freq_quad)
    quad_data2 = scaler3.transform(freq_quad)
else:
    quad_data2 = freq_quad
# Instantiate model with 1000 decision trees
quad_rf = RandomForestRegressor(n_estimators = 9999, random_state = 42)
# Train the model on training data
quad_rf.fit(quad_data2, quad_ratings)

RandomForestRegressor(n_estimators=9999, random_state=42)

Quadrant Random Forest Validation

In [ ]:
from os import listdir
from os.path import isfile, join
from sklearn.metrics import mean_squared_error

def predictQuadRF(x):
    d = Data(directory=x)
    raw_data = d.filter_confidence()
    #print(raw_data)
    filename_2 = [f for f in listdir(x) if isfile(join(x, f))]#[0]
    predictions = []
    for filename in filename_2:
        shape = cv2.imread(x + str(filename)).shape
        temp = list(raw_data[filename.replace(".png","")])
        for i in range(len(temp)):
            element = temp[i]
            temp_data_holder = [{},{},{},{}]
            temp_data_holder_2 = []
            temp_image_name = image_names[i]
            temp_image = cv2.imread(directory + temp_image_name + '.png')
            w = temp_image.shape[0]
            h = temp_image.shape[1]
            #print(temp)
            center_X = (int(element[0])+int(element[2]))/2
            center_Y = (int(element[1])+int(element[3]))/2
            if center_X >= w/2 and center_Y >= h/2:
                if temp_data_holder[0].get(int(element[5])) == None:
                    temp_data_holder[0][int(element[5])] = 1
                else:
                    temp_data_holder[0][int(element[5])] = temp_data_holder[0][int(element[5])] + 1
            if center_X < w/2 and center_Y >= h/2:
                if temp_data_holder[1].get(int(element[5])) == None:
                    temp_data_holder[1][int(element[5])] = 1
                else:
                    temp_data_holder[1][int(element[5])] = temp_data_holder[1][int(element[5])] + 1
            if center_X < w/2 and center_Y < h/2:
                if temp_data_holder[2].get(int(element[5])) == None:
                    temp_data_holder[2][int(element[5])] = 1
                else:
                    temp_data_holder[2][int(element[5])] = temp_data_holder[2][int(element[5])] + 1
            if center_X >= w/2 and center_Y < h/2:
                if temp_data_holder[3].get(int(element[5])) == None:
                    temp_data_holder[3][int(element[5])] = 1
                else:
                    temp_data_holder[3][int(element[5])] = temp_data_holder[3][int(element[5])] + 1
        for quadrant in temp_data_holder:
            for i in range(8):
                if quadrant.get(i) == None:
                    temp_data_holder_2.append(0)
                else:
                    temp_data_holder_2.append(quadrant[i])
        if COLOR:
            c = colorfulness()
            temp_data_holder_2.append(c[int(temp_image_name)])
        try:
            prediction = quad_rf.predict([temp_data_holder_2])
            predictions.append(prediction[0])
        except Exception as e:
            print("Something went wrong hehe :)")
    return predictions
    

predicted = predictQuadRF("small_test/")

print(predicted)
data = {}
path = r"dataset\preprocess\train_means_list.csv"
with open(path) as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if "english_resized" in row[0]:
            key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
            value = float(row[1])
            data[key] = value
actual = []
#print(data)
files = [f for f in listdir("small_test/") if isfile(join("small_test/", f))]
for f in files:
    if not data.get(int(f.replace(".png",""))) == None:
        actual.append(data[int(f.replace(".png",""))])
    else:
        actual.append(-1)
print(actual)

predicted_2 = []
actual_2 = []
for i in range(len(actual)):
    if not actual[i] == -1:
        try:
            predicted_2.append(predicted[i])
            actual_2.append(actual[i])
        except Exception as e:
            print("something went wrong again :(")

mean_squared_error(actual_2, predicted_2)

Using cache found in C:\Users\woprg/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-18 Python-3.10.4 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Something went wrong hehe :)
Something went wrong hehe :)
Something went wrong hehe :)
[4.451114020488926, 4.4291839167451945, 4.451114020488926, 4.3786502155318585, 4.3612535388185965, 4.524538100499158, 4.428830694058246, 4.477704631293092, 4.440203535435431, 4.438034889572963, 4.3786502155318585, 4.444365203777698, 4.380669121622096, 4.381801867343205, 4.444365203777698, 4.3786502155318585, 4.451114020488926, 4.381801867343205, 4.262593255678566, 4.451114020488926, 4.524538100499158, 4.428830694058246, 4.524538100499158, 4.451114020488926, 4.451114020488926, 4.444365203777698, 4.438034889572963, 4.436158909741406, 4.477704631293092, 4.4445707778930625, 4.440203535435431, 4.4445707778930625, 4.477704631293092, 4.451114020488926, 4.436158909741406, 4.477704631293092, 4.451114020488926, 4.477704631293092, 4.380669121622096, 4.380669121622096, 4.524538100499158, 4.477704631293092, 4.4291839167451945, 4.451114020488926, 4.380669121622096, 4.400908756618148, 4.451114020488926, 4.400908756

1.1829096816036755

Average Position Data Formatting

In [21]:
import csv
import numpy as np

path = r"dataset\preprocess\train_means_list.csv"

SVM = True
COLOR = False

normalizer = tf.keras.layers.Normalization(axis=-1)

data = {}

with open(path) as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if "english_resized" in row[0]:
            key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
            value = float(row[1])
            data[key] = value

avg_pos_temp = avgPos()
avg_pos = []
ratings = []
if COLOR:
    c = colorfulness()

for key, element in avg_pos_temp.items():
    if not data.get(key) == None:
        temp_data = []
        shape = cv2.imread(directory + str(key) + ".png").shape
        #print(shape)
        for i in range(8):
            if element.get(i) == None:
                temp_data.append(0)
                temp_data.append(0)
            else:
                temp_data.append(element[i][0]
                #/shape[1]
                )
                temp_data.append(element[i][1]
                #/shape[0]
                )
        if COLOR:
            temp_data.append(c[int(key)])
        if SVM:
            avg_pos.append(temp_data)
            if data[key] >= 4.5:
                ratings.append(1)
            else:
                ratings.append(0)
        else:
            avg_pos.append(np.asarray(temp_data).astype(np.float32))
            ratings.append((data[key]))

#print(avg_pos)
#print(ratings)

if not SVM:
    normalizer.adapt(tf.convert_to_tensor(avg_pos))

    linear_model = tf.keras.Sequential([
        normalizer,
        layers.Dense(units=1)
    ])

    linear_model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
        loss='mean_squared_logarithmic_error',
        metrics=[tf.keras.metrics.MeanSquaredLogarithmicError()])

    history = linear_model.fit(
        tf.convert_to_tensor(avg_pos),
        tf.convert_to_tensor(ratings),
        epochs=100,
        validation_split=0.2,
        # Suppress logging.
        verbose=0)



SVM

In [22]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler

apply_scaler = True

#print(avg_pos)

if apply_scaler:
    scaler1 = StandardScaler()
    scaler1.fit(avg_pos)
    data = scaler1.transform(avg_pos)
else:
    data = avg_pos

clf = svm.SVC()

clf.fit(data, ratings)

SVC()

Validate SVM

In [24]:
from os import listdir
from os.path import isfile, join

COLOR = False

def predictSVM(x):
    if COLOR:
        c = colorfulness()
    d = Data(directory=x)
    raw_data = d.filter_confidence()
    #print(raw_data)
    filename_2 = [f for f in listdir(x) if isfile(join(x, f))]#[0]
    predictions = []
    for filename in filename_2:
        shape = cv2.imread(x + str(filename)).shape
        temp = list(raw_data[filename.replace(".png","")])
        for i in range(len(temp)):
            element = temp[i]
            avg_pixel_position_for_class = {}
            count = {}
            if avg_pixel_position_for_class.get(int(element[5])) == None:
                avg_pixel_position_for_class[int(element[5])] = ((float(element[0])+float(element[2]))/2,(float(element[1])+float(element[3]))/2)
                count[int(element[5])] = 1
            else:
                count[int(element[5])] += 1
                avg_pixel_position_for_class[int(element[5])] = ((((float(element[0])+float(element[2]))/2)+avg_pixel_position_for_class[int(element[5])][0])/count[int(element[5])],(((float(element[1])+float(element[3]))/2)+avg_pixel_position_for_class[int(element[5])][0])/count[int(element[5])])
        processed_data = []
        for i in range(8):
            if avg_pixel_position_for_class.get(i) == None:
                processed_data.append(0)
                processed_data.append(0)
            else:
                processed_data.append(avg_pixel_position_for_class[i][0]
                #/shape[1]
                )
                processed_data.append(avg_pixel_position_for_class[i][1]
                #/shape[0]
                )
        if COLOR:
            processed_data.append(c[int(filename.replace(".png",""))])

        prediction = clf.predict([processed_data])
        predictions.append(prediction[0])
    return predictions

predicted = predictSVM("small_test/")

#print(predicted)
data = {}
path = r"dataset\preprocess\train_means_list.csv"
with open(path) as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if "english_resized" in row[0]:
            key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
            value = float(row[1])
            data[key] = value
actual = []
#print(data)
files = [f for f in listdir("small_test/") if isfile(join("small_test/", f))]
for f in files:
    if not data.get(int(f.replace(".png",""))) == None:
        act_rating = data[int(f.replace(".png",""))]
        if act_rating >= 4:
            actual.append(1)
        else:
            actual.append(0)
    else:
        actual.append(-1)
#print(actual)

predicted_2 = []
actual_2 = []
for i in range(len(actual)):
    if not actual[i] == -1:
        try:
            predicted_2.append(predicted[i])
            actual_2.append(actual[i])
        except Exception as e:
            print("something went wrong again :(")

print(predicted_2)
print(actual_2)

Using cache found in C:\Users\woprg/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-18 Python-3.10.4 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Validate Average Position Model

In [45]:
from os import listdir
from os.path import isfile, join
from sklearn.metrics import mean_squared_error

def predict(x):
    d = Data(directory=x)
    raw_data = d.filter_confidence()
    #print(raw_data)
    filename_2 = [f for f in listdir(x) if isfile(join(x, f))]#[0]
    predictions = []
    for filename in filename_2:
        temp_image_name = image_names[i]
        shape = cv2.imread(x + str(filename)).shape
        temp = list(raw_data[filename.replace(".png","")])
        for i in range(len(temp)):
            element = [float(x) for x in list(temp[i])]
            avg_pixel_position_for_class = {}
            #print(avg_pixel_position_for_class.get(int(element[5])))
            count = {}
            if avg_pixel_position_for_class.get(int(element[5])) == None:
                avg_pixel_position_for_class[int(element[5])] = ((float(element[0])+float(element[2]))/2,(float(element[1])+float(element[3]))/2)
                count[int(element[5])] = 1
            else:
                count[int(element[5])] += 1
                avg_pixel_position_for_class[int(element[5])] = ((((float(element[0])+float(element[2]))/2)+avg_pixel_position_for_class[int(element[5])][0])/count[int(element[5])],(((float(element[1])+float(element[3]))/2)+avg_pixel_position_for_class[int(element[5])][0])/count[int(element[5])])
            processed_data = []
            for i in range(8):
                if avg_pixel_position_for_class.get(i) == None:
                    processed_data.append(0)
                    processed_data.append(0)
                else:
                    processed_data.append(avg_pixel_position_for_class[i][0]
                    #/shape[1]
                    )
                    processed_data.append(avg_pixel_position_for_class[i][1]
                    #/shape[0]
                    )
            #print(processed_data)
            if COLOR:
                c = colorfulness()
                processed_data.append(c[int(temp_image_name)])
            prediction = linear_model.predict([processed_data])
            predictions.append(prediction[0][0])
    return predictions
    
predicted = predict("small_test/")
actual = []
#print(data)
files = [f for f in listdir("small_test/") if isfile(join("small_test/", f))]
for f in files:
    if not data.get(int(f.replace(".png",""))) == None:
        actual.append(data[int(f.replace(".png",""))])
    else:
        actual.append(-1)
#print(actual)

predicted_2 = []
actual_2 = []
for i in range(len(actual)):
    if not actual[i] == -1:
        try:
            predicted_2.append(predicted[i])
            actual_2.append(actual[i])
        except Exception as e:
            print("something went wrong again :(")

mean_squared_error(actual_2, predicted_2)
#[    0.77215     0.69101     0.83729      0.1006     0.12892    0.044708           0           0           0           0           0           0, 0.73748, 0.036506, 0.45679, 0.69613]

Using cache found in C:\Users\woprg/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-18 Python-3.10.4 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


None
1/1 [==============================] - 0s 31ms/step
None
1/1 [==============================] - 0s 23ms/step
None
1/1 [==============================] - 0s 25ms/step
None
1/1 [==============================] - 0s 23ms/step
None
1/1 [==============================] - 0s 25ms/step
None
1/1 [==============================] - 0s 23ms/step
None
1/1 [==============================] - 0s 25ms/step
None
1/1 [==============================] - 0s 26ms/step
None
1/1 [==============================] - 0s 23ms/step
None
1/1 [==============================] - 0s 26ms/step
None
1/1 [==============================] - 0s 24ms/step
None
1/1 [==============================] - 0s 23ms/step
None
1/1 [==============================] - 0s 24ms/step
None
1/1 [==============================] - 0s 23ms/step
None
1/1 [==============================] - 0s 25ms/step
None
1/1 [==============================] - 0s 25ms/step
None
1/1 [==============================] - 0s 24ms/step
None
1/1 [=====================

1.2943198967026341

Get Ratings

In [25]:
import csv
import numpy as np

path = r"dataset\preprocess\train_means_list.csv"

SVM = False
RANDOMFOREST = True

data = {}

with open(path) as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if "english_resized" in row[0]:
            key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
            value = float(row[1])
            data[key] = value

class_freq_temp = freqClass()

#print(class_freq_temp)

data_2 = {}

class_freq = {}

for key, value in class_freq_temp.items():
    if not data.get(key) == None:
        temp_val = {0:[],1:[],2:[],3:[],4:[],5:[],6:[],7:[]}
        for i in range(8):
            if value.get(i) == None:
                temp_val[i] = 0
            else:
                temp_val[i] = value[i]
        class_freq[key] = temp_val

for key in list(class_freq_temp.keys()):
    if not data.get(key) == None:
        data_2[key] = data[key]

train_features = list(class_freq.copy().values())
train_labels = list(data_2.copy().values())

if not SVM:
    if not RANDOMFOREST:
        for i in range(len(train_features)):
            train_features[i] = np.asarray(list(train_features[i].values())).astype(np.float32)

        train_features = tf.convert_to_tensor(train_features)
        train_labels = tf.convert_to_tensor(np.asarray(train_labels).astype(np.float32))
    else:
        for i in range(len(train_features)):
            train_features[i] = list(train_features[i].values())
else:
    for i in range(len(train_features)):
        train_features[i] = list(train_features[i].values())
    for i in range(len(train_labels)):
        if train_labels[i]>=4:
            train_labels[i]=1
        else:
            train_labels[i]=0

#print(train_features)
#print(train_labels)



Random Forest

In [26]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

apply_scaler3 = True

if apply_scaler3:
    scaler3 = StandardScaler()
    scaler3.fit(train_features)
    data3 = scaler3.transform(train_features)
else:
    data3 = train_features
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(data3, train_labels)

RandomForestRegressor(n_estimators=1000, random_state=42)

Random Forest Evaluator

In [28]:
from os import listdir
from os.path import isfile, join

def predictRandomForest2(x):
    d = Data(directory=x)
    raw_data = d.filter_confidence()
    #print(raw_data)
    filename_2 = [f for f in listdir(x) if isfile(join(x, f))]#[0]
    predictions = []
    for filename in filename_2:
        shape = cv2.imread(x + str(filename)).shape
        temp = list(raw_data[filename.replace(".png","")])
        formatted_data = []
        for i in range(len(temp)):
            element = temp[i]
            per_image_data = {}
            if per_image_data.get(int(element[5])) == None:
                per_image_data[int(element[5])] = 1
            else:
                per_image_data[int(element[5])] = per_image_data[int(element[5])] + 1
        for i in range(8):
            if per_image_data.get(i) == None:
                formatted_data.append(0)
            else:
                formatted_data.append(per_image_data[i])
        predictions.append(float(rf.predict([formatted_data])[0]))
    return predictions

predicted = predictRandomForest2("small_test/")
actual = []
#print(data)
files = [f for f in listdir("small_test/") if isfile(join("small_test/", f))]
for f in files:
    if not data.get(int(f.replace(".png",""))) == None:
        actual.append(data[int(f.replace(".png",""))])
    else:
        actual.append(-1)
#print(actual)

predicted_2 = []
actual_2 = []
for i in range(len(actual)):
    if not actual[i] == -1:
        try:
            predicted_2.append(predicted[i])
            actual_2.append(actual[i])
        except Exception as e:
            print("something went wrong again :(")

mean_squared_error(actual_2, predicted_2)

Using cache found in C:\Users\woprg/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-18 Python-3.10.4 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


1.1662420227258388

SVM

In [34]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler

apply_scaler2 = True

if apply_scaler2:
    scaler2 = StandardScaler()
    scaler2.fit(train_features)
    data2 = scaler2.transform(train_features)
else:
    data2 = train_features

clf = svm.SVC()

clf.fit(data2, train_labels)

SVC()

SVM Predict

In [35]:
from os import listdir
from os.path import isfile, join

def predictSVM2(x):
    d = Data(directory=x)
    filename = [f for f in listdir(x) if isfile(join(x, f))][0]
    raw_data = d.filter_confidence()
    temp = list(raw_data.values())
    shape = cv2.imread(x + str(filename)).shape
    formatted_data = []
    for i in range(len(temp)):
        per_image_data = {}
        for element in temp[i]:
            if per_image_data.get(int(element[5])) == None:
                per_image_data[int(element[5])] = 1
            else:
                per_image_data[int(element[5])] = per_image_data[int(element[5])] + 1
        for i in range(8):
            if per_image_data.get(i) == None:
                formatted_data.append(0)
            else:
                formatted_data.append(per_image_data[i])
    return float(clf.predict([formatted_data])[0])


print(predictSVM2("small_test/"))
data = {}
path = r"dataset\preprocess\train_means_list.csv"
with open(path) as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if "english_resized" in row[0]:
            key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
            value = float(row[1])
            data[key] = value
print(data[int([f for f in listdir("small_test/") if isfile(join("small_test/", f))][0].replace(".png",""))])

Using cache found in C:\Users\woprg/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-18 Python-3.10.4 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


1.0
1.892075078206465


Model

In [31]:
linear_model = tf.keras.Sequential([
    layers.Dense(units=1)
])

linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_squared_logarithmic_error',
    metrics=[tf.keras.metrics.MeanSquaredError()])

print(len(list(train_features)))
print(len(train_labels))

history = linear_model.fit(
    train_features,
    train_labels,
    epochs=100,
    validation_split=0.2,
    # Suppress logging.
    verbose=1)

255
255
Epoch 1/100
7/7 [==============================] - 0s 16ms/step - loss: 0.6191 - mean_squared_error: 27.8766 - val_loss: 0.4000 - val_mean_squared_error: 17.3155
Epoch 2/100
7/7 [==============================] - 0s 5ms/step - loss: 0.2709 - mean_squared_error: 7.7031 - val_loss: 0.2892 - val_mean_squared_error: 12.7148
Epoch 3/100
7/7 [==============================] - 0s 5ms/step - loss: 0.2680 - mean_squared_error: 10.3328 - val_loss: 0.2458 - val_mean_squared_error: 10.9122
Epoch 4/100
7/7 [==============================] - 0s 6ms/step - loss: 0.1544 - mean_squared_error: 3.9679 - val_loss: 0.1353 - val_mean_squared_error: 3.2028
Epoch 5/100
7/7 [==============================] - 0s 6ms/step - loss: 0.1134 - mean_squared_error: 3.3192 - val_loss: 0.1205 - val_mean_squared_error: 3.6629
Epoch 6/100
7/7 [==============================] - 0s 6ms/step - loss: 0.0938 - mean_squared_error: 2.5317 - val_loss: 0.1350 - val_mean_squared_error: 2.7089
Epoch 7/100
7/7 [===============

In [32]:
img_path = "small_test/"

def getPrediction(imgLink):
    d = Data(directory=imgLink)
    raw_data = d.filter_confidence()
    temp = list(raw_data.values())
    for i in range(len(temp)):
        per_image_data = {}
        for element in temp[i]:
            if per_image_data.get(int(element[5])) == None:
                per_image_data[int(element[5])] = 1
            else:
                per_image_data[int(element[5])] = per_image_data[int(element[5])] + 1
    processed_data = []
    for i in range(8):
        if per_image_data.get(i) == None:
            processed_data.append(0)
        else:
            processed_data.append(per_image_data[i])
    prediction = linear_model.predict([processed_data])
    return prediction

    
print(getPrediction(img_path))
data = {}
path = r"dataset\preprocess\train_means_list.csv"
with open(path) as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if "english_resized" in row[0]:
            key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
            value = float(row[1])
            data[key] = value
print(data[int([f for f in listdir("small_test/") if isfile(join("small_test/", f))][0].replace(".png",""))])

Using cache found in C:\Users\woprg/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-18 Python-3.10.4 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


1/1 [==============================] - 0s 42ms/step
[[     3.9247]]
3.61271676300578


Feature Mixer

In [1]:
def mixFeatures(avgPosition,quadrants,classfreq,modeltype):
    import csv
    path = r"dataset\preprocess\train_means_list.csv"
    data = {}
    with open(path) as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            if "english_resized" in row[0]:
                key = int(row[0][row[0].rfind("/")+1:row[0].rfind(".")])
                value = float(row[1])
                data[key] = value
    training = {}
    a = []
    b = []
    def createSVM(training, labels, apply_scaler2=True):
        from sklearn import svm
        from sklearn.preprocessing import StandardScaler

        if apply_scaler2:
            scaler2 = StandardScaler()
            scaler2.fit(training)
            data2 = scaler2.transform(training)
        else:
            data2 = training

        clf = svm.SVC()

        clf.fit(data2, labels)
        return clf
    def createLin(training, labels, normalize=True):
        if normalize:
            normalizer = tf.keras.layers.Normalization(axis=-1)
            normalizer.adapt(tf.convert_to_tensor(training))
            linear_model = tf.keras.Sequential([
                normalizer,
                layers.Dense(units=1)
            ])
        else:
            linear_model = tf.keras.Sequential([
                layers.Dense(units=1)
            ])

        linear_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
            loss='mean_squared_logarithmic_error',
            metrics=[tf.keras.metrics.MeanSquaredError()])

        history = linear_model.fit(
            tf.convert_to_tensor(training),
            tf.convert_to_tensor(labels),
            epochs=100,
            validation_split=0.2,
            # Suppress logging.
            verbose=1)
        return linear_model
    def createRF(training, labels, apply_scaler3=True):
        # Import the model we are using
        from sklearn.ensemble import RandomForestRegressor
        from sklearn.preprocessing import StandardScaler

        if apply_scaler3:
            scaler3 = StandardScaler()
            scaler3.fit(training)
            data3 = scaler3.transform(training)
        else:
            data3 = training
        # Instantiate model with 1000 decision trees
        rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
        # Train the model on training data
        rf.fit(data3, labels)
        return rf
    def formatAvgPos(SVM=False,COLOR=False):
        import numpy as np

        avg_pos_temp = avgPos()
        avg_pos = {}
        ratings = []
        if COLOR:
            c = colorfulness()

        for key, element in avg_pos_temp.items():
            if not data.get(key) == None:
                temp_data = []
                shape = cv2.imread(directory + str(key) + ".png").shape
                #print(shape)
                for i in range(8):
                    if element.get(i) == None:
                        temp_data.append(0)
                        temp_data.append(0)
                    else:
                        temp_data.append(element[i][0]
                        #/shape[1]
                        )
                        temp_data.append(element[i][1]
                        #/shape[0]
                        )
                if COLOR:
                    temp_data.append(c[int(key)])
                if SVM:
                    avg_pos.append(temp_data)
                    if data[key] >= 4.5:
                        ratings.append(1)
                    else:
                        ratings.append(0)
                else:
                    avg_pos[int(key)] = temp_data
                    ratings.append((data[key]))
        return avg_pos,ratings
    def formatFreq(SVM=False,RANDOMFOREST=False):
        import numpy as np

        class_freq_temp = freqClass()

        #print(class_freq_temp)

        data_2 = {}

        class_freq = {}

        for key, value in class_freq_temp.items():
            if not data.get(key) == None:
                temp_val = {0:[],1:[],2:[],3:[],4:[],5:[],6:[],7:[]}
                for i in range(8):
                    if value.get(i) == None:
                        temp_val[i] = 0
                    else:
                        temp_val[i] = value[i]
                class_freq[key] = temp_val

        for key in list(class_freq_temp.keys()):
            if not data.get(key) == None:
                data_2[key] = data[key]

        return class_freq, data_2
    def formatQuad(SVM=False, COLOR=False):
        import numpy as np

        if COLOR:
            c = colorfulness()

        freq_quad_temp = freqQuad()
        freq_quad = {}
        quad_ratings = []
        quad_ratings_binary = []

        for key, value in freq_quad_temp.items():
            if not data.get(key) == None:
                if COLOR:
                    value.append(c[key])
                freq_quad[key] = value
                quad_ratings.append(data[key])
                if data[key] >= 4:
                    quad_ratings_binary.append(1)
                else:
                    quad_ratings_binary.append(0)
        if SVM:
            return freq_quad, quad_ratings_binary
        else:
            return freq_quad, quad_ratings
    def integrateData(input):
        for key, value in input.items():
            if training.get(key) == None:
                training[key] = value
            else:
                training[key].extend(value)
    def makeTrainTest(SVM=False, threshold=4):
        for key, value in training.items():
            a.append(value)
            if SVM:
                data_index = data[key]
                if data_index > threshold:
                    b.append(1)
                else:
                    b.append(0)
            else:
                b.append(data[key])
    integrateData(formatAvgPos()[0])
    integrateData(formatFreq()[0])
    integrateData(formatQuad()[0])
    makeTrainTest(SVM=False)
    return createRF(a,b)

print(mixFeatures(1,2,3,4))


FileNotFoundError: [Errno 2] No such file or directory: 'dataset\\preprocess\\train_means_list.csv'